In [4]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
from codes.util import *
from PIL import Image
import matplotlib.image as mpimg
from dataset import get_label
from codes.util.check_dir import mk_dir


In [5]:
def iou(a, b):    
    a_norm = np.sqrt(np.sum(a**2))
    b_norm = np.sqrt(np.sum(b**2))
    a = (a / a_norm).reshape(-1)
    b = (b / b_norm).reshape(-1)
    up = [min(a[i], b[i]) for i in range(a.shape[0])]
    down = [max(a[i], b[i]) for i in range(a.shape[0])]   
    if sum(down)  != 0:
        return sum(up) / sum(down)
    else:
        return -1
def block_ptb(ptb, k=8, norm=2):    
    res = np.zeros((k,k))
    for r in range(k):
        for c in range(k):   
            tmp = ptb[:,12*r:12*(r+1),12*c:12*(c+1)]
            if norm == 2:                
                res[r][c] = np.sqrt(np.sum(tmp**2))
            else:
                res[r][c] = np.abs(tmp.sum(0)).max()
    return res.reshape(-1)
def cos(a, b):
    up = np.sum(a*b)
    down = np.linalg.norm(a) * np.linalg.norm(b)
    return up/down

In [11]:
data = 'voc'
net = 'alexnet'
cates = os.listdir(f'./experiment/attribution/Perturb/li/{net}/{data}')

for cate in cates:
    for img in os.listdir(f'./experiment/attribution/Perturb/li/{net}/{data}/{cate}'):  
        info = {}
        for i,p in enumerate(['l2', 'li']):
            info[p] = {}           
            croot = f'./experiment/attribution/CSV/{p}/{net}/{data}/{cate}/{img}/shap.csv' 
            proot = f'./experiment/attribution/Perturb/{p}/{net}/{data}/{cate}/{img}/ptb.npy'
            
            if not (os.path.exists(croot) and os.path.exists(proot)):                
                continue 
    
            ptb = np.load(proot).squeeze()[:,8:104, 8:104]                  
            mag = block_ptb(ptb,norm=2)              
            shap = -1*np.genfromtxt(croot, delimiter=',')[-1,:].reshape(-1)  
        
            info[p]['mag'] = mag
            info[p]['shap'] = shap
        mk_dir(f'./experiment/attribution/iou/{data}_{net}')
        np.save(f'./experiment/attribution/iou/{data}_{net}/{img[:-4]}.npy', info)
        print (cate, img, 'saved')           

0 2008_008570.jpg saved


In [14]:

data = 'sdd'
net = 'vgg16'
def cal_iou(data, net):
    shap, mag = [], []
    for img in os.listdir(f'./experiment/attribution/iou/{data}_{net}'):
        p = f'./experiment/attribution/iou/{data}_{net}/{img}'
        f = np.load(p,allow_pickle=True).item()
        try:
            l2_shap = f['l2']['shap']
            l2_mag = f['l2']['mag']
            li_mag = f['li']['mag']
            li_shap = f['li']['shap']
        
            nor_l2_shap = (l2_shap-l2_shap.min()) / (l2_shap.max()-l2_shap.min())
            nor_l2_mag = (l2_mag-l2_mag.min()) / (l2_mag.max()-l2_mag.min())
            nor_li_shap = (li_shap-li_shap.min()) / (li_shap.max()-li_shap.min())
            nor_li_mag = (li_mag-li_mag.min()) / (li_mag.max()-li_mag.min())

   
            shap_iou, mag_iou = iou(nor_l2_shap, nor_li_shap), iou(nor_l2_mag, nor_li_mag)
            if shap_iou>0 and mag_iou>0:
                shap.append(shap_iou)
                mag.append(mag_iou)        
        except:
            continue     
    shap = np.array(shap)        
    mag = np.array(mag)
    print (data, net, len(shap), len(mag))
    print (shap.mean(), shap.std(), mag.mean(), mag.std())

# print (sum(shap)/len(shap), sum(mag)/len(mag))

In [16]:

for net in ['alexnet']:#, 'alexnet', ]:
    for data in ['voc','cub','sdd']:
        cal_iou(data, net)

voc alexnet 1 1
0.7917026840049256 0.0 0.6181878822962854 0.0


FileNotFoundError: [Errno 2] No such file or directory: './experiment/attribution/iou/cub_alexnet'